In [39]:
import os
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer, util
import pickle
import copy


In [4]:
grade = "3"
target_path = f"../../data/question_correctness/.csv"
scores_path = "../../Gates.ReadComp_By-Item_Gr3-5(CM).xlsx"
corpus_path = f"../../subtest_txt/gr{grade}_paragraphs.txt"
questions_path = f"../../subtest_txt/gr{grade}_questions.txt"

In [5]:
sub_tests = []
with open(corpus_path,'r') as fp:
    sub_test = fp.readline()
    while sub_test:
        sub_tests.append(sub_test)
        sub_test = fp.readline()


df = pd.read_excel(scores_path)
sub_test_number = df.columns[1:]

print(sub_test_number)

questions_ranges = [(1,5), (6,8), (9,13), (14,16), (17,21), (22,27), (28,30),
                   (31,35), (36,40), (41,43), (44,48)]

model = SentenceTransformer('paraphrase-mpnet-base-v2')

#Change the length to 200
model.max_seq_length = 500

sub_tests_embed = model.encode(sub_tests, show_progress_bar=True)

print(sub_tests_embed.shape)

Index(['Gr3.RC.Gates_01', 'Gr3.RC.Gates_02', 'Gr3.RC.Gates_03',
       'Gr3.RC.Gates_04', 'Gr3.RC.Gates_05', 'Gr3.RC.Gates_06',
       'Gr3.RC.Gates_07', 'Gr3.RC.Gates_08', 'Gr3.RC.Gates_09',
       'Gr3.RC.Gates_10',
       ...
       'Gr5.RC.Gates_45', 'Gr5.RC.Gates_46', 'Gr5.RC.Gates_47',
       'Gr5.RC.Gates_48', 'Gr5.RC.Gates_RawScore', 'Gr5.RC.Gates_GradeEquiv',
       'Gr5.RC.Gates_NCE', 'Gr5.RC.Gates_NPR', 'Gr5.RC.Gates_NS',
       'Gr5.RC.Gates_ESS'],
      dtype='object', length=162)



(12, 768)


In [11]:
import nltk
from nltk.parse import CoreNLPParser
import statistics

def get_average_depth(parser,text):
    sentences = nltk.sent_tokenize(text)
#     print(sentences)
    depths = []
    for s in sentences:
        parse = next(parser.raw_parse(s))
#         parse.draw()
        depths.append(parse.height())
        
    return statistics.mean(depths)

def get_avg_stdev_depth(parser,text):
    sentences = nltk.sent_tokenize(text)
#     print(sentences)
    depths = []
    for s in sentences:
        parse = next(parser.raw_parse(s))
#         parse.draw()
        depths.append(parse.height())
        
    return [statistics.mean(depths), statistics.stdev(depths)]

parser = CoreNLPParser(url='http://localhost:9000')

# parse = next(parser.raw_parse(nltk.sent_tokenize(sub_tests[7])[4]))
# print(parse.height())
# parse.draw()



In [12]:
depths = []
for text in sub_tests:
    depths.append(get_avg_stdev_depth(parser,text))

print(depths)

[[7.666666666666667, 1.3662601021279464], [9, 2.0], [8.75, 3.1959796173138706], [8, 1.3416407864998738], [7.375, 1.9226098333849673], [8.235294117647058, 2.305683514836378], [10.333333333333334, 3.9619401430321606], [8, 3.2145502536643185], [9.571428571428571, 2.14919697074224], [8.071428571428571, 2.234839031005476], [8.75, 1.9086270308410553], [7.181818181818182, 2.238970078627004]]


In [8]:
questions = []
answers = []

with open(questions_path,'r') as fp:
    for _ in range(48):
        question = fp.readline()
        answer = fp.readline()
#         print(question,answer)
        questions.append(question)
        answers.append(answer)
        
print(len(questions))
print(len(answers))

questions_embed = model.encode(questions, show_progress_bar=True)
answers_embed = model.encode(answers, show_progress_bar=True)

print(questions_embed.shape)
print(answers_embed.shape)

48
48



(48, 768)
(48, 768)


In [13]:
questions_depth = []
answer_depth = []

for q in questions:
    questions_depth.append(get_average_depth(parser,q))

for a in answers:
    answer_depth.append(get_average_depth(parser,a))

In [14]:
feature_path = "../../SoR_Alberta.Shared.Data.and.Codebook.xlsx"#"../data/gr3/gr3_features.xlsx"
feature_names = ['G3.PPVT.Vocab.raw',
                 'G3.Elision.PA.raw',
                 'G3.Syn.GramCorrect.raw',
                 'G3.TOWRE.SWE.raw',
                 'G3.TOWRE.PDE.raw',
                 'G3.WordID.raw',
                 'G3.OL.Spell.Total',
                 'G3.OL.OrthoChoice.1.2.Total',
                 'G3.DigitSpan.raw',]


In [43]:
data = []
df2 = pd.read_excel(feature_path)

for i in df.index:
    is_available = True
    skills = []
    for name in feature_names:
        value = df2[name][i]
        if value < 0:
            is_available = False
            break
        skills.append(value)
    if not is_available:
        continue

    for index, q_range in enumerate(questions_ranges):
        entry = []
        entry.append(skills+depths[index])
        entry.append(sub_tests_embed[index])
        for j in range(q_range[0],q_range[1]+1):
            # add embeddings
            detail = []
            detail.append(questions_embed[j-1])
            detail.append(answers_embed[j-1])
            # add cfg depth
            new_entry = copy.deepcopy(entry)

            new_entry[0] += [questions_depth[j-1],answer_depth[j-1]]
            new_entry[0] = np.asarray(new_entry[0])
            
            if df[f"Gr{grade}.RC.Gates_"+"{:02d}".format(j)][i] == 1:
                detail.append(1)
            elif df[f"Gr{grade}.RC.Gates_"+"{:02d}".format(j)][i] in (2,0):
                detail.append(0)
            else:
                print('DNE')
            data.append(new_entry+detail)

print(data[0][0])
print(data[1][0])
    

[33.         26.          5.         60.         22.         59.
  8.         21.         11.          7.66666667  1.3662601   7.
  4.        ]
[33.         26.          5.         60.         22.         59.
  8.         21.         11.          7.66666667  1.3662601   6.
  6.        ]


In [44]:
with open("data.pkl",'wb') as fp:
    pickle.dump(data, fp)


In [45]:
with open("data.pkl",'rb') as fp:
    new_data = pickle.load(fp)

In [46]:
print(len(new_data[1][0]))
# print(len(new_data))

13
